In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style("darkgrid")
import datetime
import calendar

from kiblib.utils.db import DbConn

In [2]:
db_conn = DbConn().create_engine()

In [3]:
year = 2025 #année pour les stats
month = 5 # Numéro du mois pour les stats

In [4]:
import locale
locale.setlocale(locale.LC_TIME,'')

'fr_FR.UTF-8'

In [5]:
month_name = calendar.month_name[month]
month_name

'mai'

## TEST COMPTAGE JOURS OUVERTS 

In [6]:
query = f"SELECT * FROM statdb.stat_entrees WHERE MONTH(`datetime`) = {month} AND YEAR(`datetime`)={year}"

In [7]:
# On importe un dataframe qui correspond au mois et à l'année rentré plus haut ("month" et "year")
comptage_entrees = pd.read_sql(query,db_conn)
comptage_entrees

,datetime,entrees
0,2025-05-02 09:00:00,77
1,2025-05-02 10:00:00,111
2,2025-05-02 11:00:00,57
3,2025-05-02 12:00:00,67
4,2025-05-02 13:00:00,99
...,...,...
221,2025-05-31 14:00:00,116
222,2025-05-31 15:00:00,164
223,2025-05-31 16:00:00,150
224,2025-05-31 17:00:00,92


In [8]:
# On transforme le datetime en date, puis on aggrège les données par date
comptage_entrees['datetime'] = pd.DatetimeIndex(comptage_entrees['datetime']).date
comptage_entrees = comptage_entrees.groupby('datetime')['entrees'].sum().reset_index()

In [9]:
# Le nombre de jours ouverts = la somme du nombre de dates où on a une info
days = len(comptage_entrees) # nombre de jours ouverts (si pas de bug du compteur d'entrées)
days

24

In [10]:
comptage_entrees

,datetime,entrees
0,2025-05-02,850
1,2025-05-03,1158
2,2025-05-04,639
3,2025-05-06,1072
4,2025-05-07,1375
5,2025-05-09,940
6,2025-05-10,1100
7,2025-05-11,504
8,2025-05-13,1105
9,2025-05-14,1517


# fin du test

In [11]:
# nb de personnes nouvellement inscrites
query = f"SELECT borrowernumber FROM koha_prod.borrowers WHERE YEAR(dateenrolled) = {year} AND MONTH(dateenrolled) = {month}"
nv_inscrits = pd.read_sql(query, db_conn)
nb_nv_inscrits = len(nv_inscrits)
nb_nv_inscrits

517

In [12]:
# nb de personnes  inscrites
query = f"SELECT COUNT(*) as inscrits FROM statdb.stat_adherents WHERE YEAR(date_extraction) = {year} AND MONTH(date_extraction) = {month}"
inscrits = pd.read_sql(query, db_conn)
nb_inscrits = inscrits['inscrits'][0]
print(nb_inscrits)


# entrées même mois un an plus tôt
query = f"SELECT COUNT(*) as inscrits FROM statdb.stat_adherents WHERE YEAR(date_extraction) = {year - 1} AND MONTH(date_extraction) = {month}"
inscrits = pd.read_sql(query, db_conn)
nb_inscrits_prec = inscrits['inscrits'][0]
print(nb_inscrits_prec)

# évolution
inscrits_evolution = round((nb_inscrits - nb_inscrits_prec) / nb_inscrits_prec * 100, 0)
print(inscrits_evolution)

15553
15168
3.0


In [13]:
# entrées
query = f"SELECT datetime, entrees FROM statdb.stat_entrees WHERE YEAR(datetime) = {year} AND MONTH(datetime) = {month}"
entrees = pd.read_sql(query, db_conn)
entrees['datetime'] = pd.to_datetime(entrees['datetime'])
nb_entrees = entrees['entrees'].sum()
print(nb_entrees)
max_entrees = entrees.groupby(entrees['datetime'].dt.date)['entrees'].sum().sort_values(ascending=False)
max_entrees_nb = max_entrees.iloc[0]
max_entrees_date = max_entrees.index[0]
print(max_entrees_nb)
print(max_entrees_date)

# entrées même mois un an plus tôt
query = f"SELECT datetime, entrees FROM statdb.stat_entrees WHERE YEAR(datetime) = {year - 1} AND MONTH(datetime) = {month}"
entrees = pd.read_sql(query, db_conn)
nb_entrees_prec = entrees['entrees'].sum()
print(nb_entrees_prec)

# évolution
entrees_evolution = round((nb_entrees - nb_entrees_prec) / nb_entrees_prec * 100, 0)
print(entrees_evolution)

24772
1517
2025-05-14
22103
12.0


In [14]:
# prets
query = f"SELECT issuedate, itemnumber FROM statdb.stat_issues WHERE YEAR(issuedate) = {year} AND MONTH(issuedate) = {month} AND location != 'MED0A'"
prets = pd.read_sql(query, db_conn)
prets['issuedate'] = pd.to_datetime(prets['issuedate'])
nb_prets = len(prets)
print(nb_prets)
max_prets = prets.groupby(prets['issuedate'].dt.date).size().sort_values(ascending=False)
max_prets_nb = max_prets.iloc[0]
max_prets_date = max_prets.index[0]
print(max_prets_nb)
print(max_prets_date)

# même mois un an plus tôt
query = f"SELECT itemnumber FROM statdb.stat_issues WHERE YEAR(issuedate) = {year - 1} AND MONTH(issuedate) = {month} AND location != 'MED0A'"
prets = pd.read_sql(query, db_conn)
nb_prets_prec = len(prets)
print(nb_prets_prec)

# évolution
prets_evolution = round((nb_prets - nb_prets_prec) / nb_prets_prec * 100, 0)
print(prets_evolution)

28179
1990
2025-05-24
26176
8.0


In [15]:
# connexions ordis
query = f"SELECT heure_deb FROM statdb.stat_webkiosk WHERE YEAR(heure_deb) = {year} AND MONTH(heure_deb) = {month}"
conn = pd.read_sql(query, db_conn)
conn['heure_deb'] = pd.to_datetime(conn['heure_deb'])
nb_conn = len(conn)
print(nb_conn)
max_conn = conn.groupby(conn['heure_deb'].dt.date).size().sort_values(ascending=False)
max_conn_nb = max_conn.iloc[0]
max_conn_date = max_conn.index[0]
print(max_conn_nb)
print(max_conn_date)

# même mois un an plus tôt
query = f"SELECT heure_deb FROM statdb.stat_webkiosk WHERE YEAR(heure_deb) = {year - 1} AND MONTH(heure_deb) = {month}"
conn = pd.read_sql(query, db_conn)
nb_conn_prec = len(conn)
print(nb_conn_prec)

# évolution
conn_evolution = round((nb_conn - nb_conn_prec) / nb_conn_prec * 100, 0)
print(conn_evolution)

5769
323
2025-05-28
4634
24.0


In [16]:
# connexions wifi
query = f"SELECT start_wifi FROM statdb.stat_wifi WHERE YEAR(start_wifi) = {year} AND MONTH(start_wifi) = {month}"
wifi = pd.read_sql(query, db_conn)
nb_wifi = len(wifi)
print(nb_wifi)

# même mois un an plus tôt
query = f"SELECT start_wifi FROM statdb.stat_wifi WHERE YEAR(start_wifi) = {year - 1} AND MONTH(start_wifi) = {month}"
wifi = pd.read_sql(query, db_conn)
nb_wifi_prec = len(wifi)
print(nb_wifi_prec)

# évolution
wifi_evolution = round((nb_wifi - nb_wifi_prec) / nb_wifi_prec * 100, 0)
print(wifi_evolution)

# retour wifi le 14/10

370
1193
-69.0


In [17]:
# réservations
query = f"SELECT reservedate FROM statdb.stat_reserves WHERE YEAR(reservedate) = {year} AND MONTH(reservedate) = {month}"
res = pd.read_sql(query, db_conn)
res['reservedate'] = pd.to_datetime(res['reservedate'])
nb_res = len(res)
print(nb_res)
max_res = res.groupby(res['reservedate'].dt.date).size().sort_values(ascending=False)
max_res_nb = max_res.iloc[0]
max_res_date = max_res.index[0]
print(max_res_nb)
print(max_res_date)

# même mois un an plus tôt
query = f"SELECT reservedate FROM statdb.stat_reserves WHERE YEAR(reservedate) = {year - 1} AND MONTH(reservedate) = {month}"
res = pd.read_sql(query, db_conn)
nb_res_prec = len(res)
print(nb_res_prec)

# évolution
res_evolution = round((nb_res - nb_res_prec) / nb_res_prec * 100, 0)
print(res_evolution)

4301
252
2025-05-07
3544
21.0


In [18]:
# site web
query = f"SELECT date, visites FROM statdb.stat_web2 WHERE YEAR(date) = {year} AND MONTH(date) = {month}"
web = pd.read_sql(query, db_conn)
nb_web = web['visites'].sum()
print(nb_web)


# même mois un an plus tôt
query = f"SELECT date, visites FROM statdb.stat_web2 WHERE YEAR(date) = {year - 1} AND MONTH(date) = {month}"
web = pd.read_sql(query, db_conn)
nb_web_prec = web['visites'].sum()
print(nb_web_prec)

# évolution
web_evolution = round((nb_web - nb_web_prec) / nb_web_prec * 100, 0)
print(web_evolution)

15209
15519
-2.0


In [19]:
# salle étude
query = f"SELECT datetime_entree FROM statdb.stat_freq_etude WHERE YEAR(datetime_entree) = {year} AND MONTH(datetime_entree) = {month}"
etude = pd.read_sql(query, db_conn)
nb_etude = len(etude)
print(nb_etude)

# même mois un an plus tôt
query = f"SELECT datetime_entree FROM statdb.stat_freq_etude WHERE YEAR(datetime_entree) = {year - 1} AND MONTH(datetime_entree) = {month}"
etude = pd.read_sql(query, db_conn)
nb_etude_prec = len(etude)
print(nb_etude_prec)

# évolution
etude_evolution = round((nb_etude - nb_etude_prec) / nb_etude_prec * 100, 0)
print(etude_evolution)

637
473
35.0


In [36]:
print(f"Chiffres de la Mouette pour le mois de {month_name} {year}")

print("="*50 +"\n")

print(f"En {month_name}, la médiathèque a été ouverte {days} jours au total.\n")

print(f"   {nb_inscrits} personnes inscrites ({int(inscrits_evolution)} %).\n")

print("En moyenne, chaque jour (par rapport au même mois de l'année précédente) :\n")

print(f"   {int(round(nb_entrees / days, 0))} entrées ({int(entrees_evolution)} %)")

#XXX prêts (-XX%)
print(f"   {int(round(nb_prets / days, 0))} prêts ({int(prets_evolution)} %)")

#XXX connexions aux ordinateurs (+XX%)
print(f"   {int(round(nb_conn / days, 0))} connexions aux ordinateurs ({int(conn_evolution)} %)")

#XXX connexions Wifi (-XX%)
print(f"   {int(round(nb_wifi / days, 0))} connexions Wifi ({int(wifi_evolution)} %)")

#XXX réservations (-XX%)
print(f"   {int(round(nb_res / days, 0))} réservations ({int(res_evolution)} %)")

#XXX visites du site internet (-XX%)
print(f"   {int(round(nb_web / days, 0))} visites du site internet ({int(web_evolution)} %)")

#XX visites en salle d'études (-XX%)
print(f"   {int(round(nb_etude / days, 0))} visites en salle d'études ({int(etude_evolution)} %)")

#XX visites du site web
print(f"   {nb_web} visites du site web ({int(web_evolution)}%)")

print("\n")
print("Les tops :")
print("="*50 +"\n")

print(f"   {max_entrees_nb} entrées le {max_entrees_date.strftime('%A %d %B %Y')}")
print(f"   {max_conn_nb} connexions aux ordinateurs le {max_conn_date.strftime('%A %d %B %Y')}")
print (f"   {max_res_nb} réservations le {max_res_date.strftime('%A %d %B %Y')}")
print(f"   {max_prets_nb} prêts le {max_prets_date.strftime('%A %d %B %Y')}")

Chiffres de la Mouette pour le mois de mai 2025

En mai, la médiathèque a été ouverte 24 jours au total.

   15553 personnes inscrites (3 %).

En moyenne, chaque jour (par rapport au même mois de l'année précédente) :

   1032 entrées (12 %)
   1174 prêts (8 %)
   240 connexions aux ordinateurs (24 %)
   15 connexions Wifi (-69 %)
   179 réservations (21 %)
   634 visites du site internet (-2 %)
   27 visites en salle d'études (35 %)
   15209 visites du site web (-2%)


Les tops :

   1517 entrées le mercredi 14 mai 2025
   323 connexions aux ordinateurs le mercredi 28 mai 2025
   252 réservations le mercredi 07 mai 2025
   1990 prêts le samedi 24 mai 2025
